<a href="https://colab.research.google.com/github/merriekay/Fantasy-Name-Generator/blob/main/Sanderson_Character_Name_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports and things
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    !pip install -q -U transformers
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

     |████████████████████████████████| 686kB 8.1MB/s 
     |████████████████████████████████| 2.5MB 8.9MB/s 
     |████████████████████████████████| 3.3MB 41.3MB/s 
     |████████████████████████████████| 901kB 41.2MB/s 


In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
#names = pd.read_csv('/content/drive/MyDrive/sanderson_names.txt',  header = None)
#names.head()

with open('/content/drive/MyDrive/sanderson_names.txt') as f:
    names = f.read()

Mounted at /content/drive


In [81]:
names = names.replace('\n'," ")
names[:100]

'Aarik Aaron Abaray Abiajan Abigail Reed Abraham Desjardins Abrial Abrobadar Abronai Abry Absence Aci'

In [82]:
# The vocabulary of our character-level language model looks like this:
"".join(sorted(set(names.lower())))

" '()-./:abcdefghijklmnopqrstuvwxyz©±√"

In [83]:
# Use Tokenizer to tokenize the Names
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(names)

In [84]:
# Embed the name 'Shallan' as tokens:
tokenizer.texts_to_sequences(["Shallan"])

[[8, 11, 2, 7, 7, 2, 5]]

In [85]:
# Revert the sequence of tokens back to the word:
tokenizer.sequences_to_texts([[8, 11, 2, 7, 7, 2, 5]])

['s h a l l a n']

In [86]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

[encoded] = np.array(tokenizer.texts_to_sequences([names])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)


for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 37) (32, 100)


In [87]:
model = keras.models.Sequential([
    keras.layers.GRU(64, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(64, return_sequences=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Epoch 1/10
492/492 [==============================] - 10s 12ms/step - loss: 2.6599
Epoch 2/10
492/492 [==============================] - 6s 11ms/step - loss: 2.3476
Epoch 3/10
492/492 [==============================] - 6s 12ms/step - loss: 2.1683
Epoch 4/10
492/492 [==============================] - 6s 12ms/step - loss: 2.0382
Epoch 5/10
492/492 [==============================] - 6s 12ms/step - loss: 1.9491
Epoch 6/10
492/492 [==============================] - 6s 12ms/step - loss: 1.8824
Epoch 7/10
492/492 [==============================] - 6s 12ms/step - loss: 1.8217
Epoch 8/10
492/492 [==============================] - 6s 12ms/step - loss: 1.7792
Epoch 9/10
492/492 [==============================] - 6s 12ms/step - loss: 1.7407
Epoch 10/10
492/492 [==============================] - 6s 12ms/step - loss: 1.7070


In [88]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

# Let's pass in 'Meredi' and see what it predicts the next letter should be according to Sanderson:
X_new = preprocess(["Meredi"])

#this line takes a look at the softmax output and returns the max
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'n'

In [89]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [90]:
tf.random.set_seed(42)

next_char("Meredi", temperature=1)

's'

In [91]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [92]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.3))

tin lishin lishir listin liston listin leshin lesha


In [93]:
print(complete_text("t", temperature=1))

tingbran mushna mortra moral morle mralinsmoshis ca


In [94]:
print(complete_text("t", temperature=2))


tpari uau duqqa d'furc litdywapi qem lim tlvidanaau


In [257]:
import random
new_name = complete_text('mer', 15, temperature=0.75)
new_name.split(" ")[0].title()

'Merli'

Next step:

Save and load the weights: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb

Deploy the model using Flask: https://github.com/mtobeiyf/keras-flask-deploy-webapp
